In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import glob
import pandas as pd

mask_directory = '/Users/kathum/Research/COPEX/sink_region_files/masks'
WAM_files = '/Volumes/LaCie_10TB_Keys_KH/era5/output/backtrack/co_altclimate_nfr_2000_2023/'
sink_region_name = 'nfr'

In [3]:
def make_mask(region):
    region_csvpath = '/Users/kathum/Research/COPEX/sink_region_files/source_region_information.csv'
    region_info = pd.read_csv(region_csvpath)
    region_filepath = region_info.loc[region_info['region_name']== region, 'mask_filepath'].values
    
    #read in the mask associated with this region
    if region.capitalize() == 'Colorado':
        mask = xr.open_dataarray(region_filepath[0])
        #remove the sink region from this mask
        remove_mask = xr.open_dataarray(region_info.loc[region_info['other_acronym']== sink_region_name, 'mask_filepath'].item())
        mask = mask - remove_mask
        mask = mask.where(mask>0, other = 0)
        remove_mask.close()
    elif region.lower() == 'western us':
        #remove colorado and the sink region from this mask
        mask = xr.open_dataarray(region_filepath[0])
        for remove_region in [region_info.loc[region_info['other_acronym']== sink_region_name, 'region_name'].item(), 'Colorado']:
            remove_mask = xr.open_dataarray(region_info.loc[region_info['region_name']== remove_region, 'mask_filepath'].item())
            mask = mask - remove_mask
        mask = mask.where(mask == 1, other = 0)
        remove_mask.close()
    elif region== 'Other Water':
        mask = 1 - xr.open_dataarray('/Users/kathum/Research/COPEX/sink_region_files/masks/land_source_region.nc') 
        mask = mask.squeeze()
        water_mask_paths = region_info.loc[(region_info['surface_type'].notnull()), ['mask_filepath']].values.squeeze()
        for path in water_mask_paths:
            remove_mask =  xr.open_dataarray(path)
            mask = mask - remove_mask
        remove_mask.close()
        mask = mask.where(mask>0, other = 0)
    elif region == 'Other Land':
        mask = xr.open_dataarray('/Users/kathum/Research/COPEX/sink_region_files/masks/land_source_region.nc')
        mask = mask.squeeze()
        land_mask_paths = region_info.loc[(region_info['surface_type'].notnull()), ['mask_filepath']].values.squeeze()
        for path in land_mask_paths:
            remove_mask =  xr.open_dataarray(path)
            mask = mask - remove_mask
        remove_mask.close()
        mask = mask.where(mask>0, other = 0)    
    else:
        mask = xr.open_dataarray(region_filepath[0])
    
    return mask

## Create CSV

In [1]:
region_csvpath = '/Users/kathum/Research/COPEX/sink_region_files/source_region_information.csv'
region_info = pd.read_csv(region_csvpath)
files = sorted(glob.glob(WAM_files+'*.nc'))[:-80][:11] #NC files to be used and removing wam output from after Sept 30th
regions = ['Other Water','Pacific Ocean','Gulf of California', 'Atlantic Ocean','Gulf of Mexico', 'Other Land', 'Western US', 'Southern US','Midwestern US','Colorado',
           'Eastern Plains','Front Range','Pikes Peak','Arkansas Basin','High Mountain Valley']

data_dict = {}
for i, file in enumerate(files):
    day_data = xr.open_dataset(file)['e_track']
    data_dict[i] = {'date':pd.date_range(file[-19:-9], periods=1)[0]}
    for region in regions:
        region_mask = make_mask(region)
        day_region_e_track = (day_data * region_mask).sum(dim=['latitude','longitude']).values
        data_dict[i][region+'_t0_e_track'] = day_region_e_track
    day_data.close()
        
data_df = pd.DataFrame(data_dict).transpose()
#show the current data in pandas dataframe
data_df.head()


NameError: name 'pd' is not defined